# Elektrostatisches Feld - Plattenkondensator

In [1]:
# Bibliotheken importieren
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt

# Farben definieren
colors = {
    "black" : "#000000",
    "U"     : "#3333cc",
    "I"     : "#e8082c",
    "R"     : "#00d000",
    "lgrey" : "#f0f0f0",
    "dgrey" : "#c0c0c0",
    "diel"  : "#99ff99",
    "white" : "#ffffff"
}

# Vorzeichnen redundanter Grafiken

# Positive Ladung
img_pos = Canvas(width=15, height=15)
img_pos.fill_style = colors["I"]
img_pos.fill_arc(7, 7, 6, 0, 2 * pi)
img_pos.stroke_style = colors["black"]
img_pos.stroke_arc(7, 7, 6, 0, 2 * pi)
img_pos.stroke_style = colors["white"]
img_pos.line_width = 2
img_pos.begin_path()
img_pos.move_to(4,  7)
img_pos.line_to(10, 7)
img_pos.stroke()
img_pos.move_to(7,  4)
img_pos.line_to(7, 10)
img_pos.stroke()

# Negative Ladung
img_neg = Canvas(width=15, height=15)
img_neg.fill_style = colors["U"]
img_neg.fill_arc(7, 7, 6, 0, 2 * pi)
img_neg.stroke_style = colors["black"]
img_neg.stroke_arc(7, 7, 6, 0, 2 * pi)
img_neg.stroke_style = colors["white"]
img_neg.line_width = 2
img_neg.begin_path()
img_neg.move_to( 4, 7)
img_neg.line_to(10, 7)
img_neg.stroke()

# Spannungsquelle
img_vs = Canvas(width=50, height=50)
img_vs.line_width = 2
img_vs.stroke_style = colors["black"]
img_vs.fill_style = colors["white"]
img_vs.fill_arc(25, 25, 24, 0, 2 * pi)
img_vs.stroke_arc(25, 25, 24, 0, 2 * pi)
# (
# leeren Kreis für Voltmeter kopieren
img_vm = Canvas(width=50, height=50)
img_vm.draw_image(img_vs)
# )
img_vs.begin_path()
img_vs.move_to(1, 25)
img_vs.line_to(49, 25)
img_vs.stroke()

# Voltmeter
img_vm.line_width = 2
img_vm.line_join = "bevel"
img_vm.begin_path()
img_vm.move_to(18, 15)
img_vm.line_to(25, 35)
img_vm.line_to(32, 15)
img_vm.stroke()

In [2]:
class Kondensator():
    
    
    
    def __init__(self, A=140, U=1, d=40, eps=1, dtype=1):
        
        layers = ["bg1",
                  "bg2",
                  "Vsource/meter",
                  "diel",
                  "cap",
                  "field",
                  "pos",
                  "neg"
                 ]
        lx = 400
        ly = 425
        
        self.dtype = dtype
        
        self.canvas = MultiCanvas(len(layers), width=lx, height=ly)
        self.c = {}
        for li in range(len(layers)):
            self.c[layers[li]] = self.canvas[li]
        
        self.drawn = {}
        
        display(self.canvas)
        
        self.U_slider = widgets.FloatSlider(min = 1,
                                            max = 2,
                                            step = 0.05,
                                            value = U,
                                            description = r"$U$")
        self.U_slider.observe(self.redraw, "value")
        self.U_unit  = widgets.Label(value="V")
        self.U_group = HBox([self.U_slider, self.U_unit])
        
        self.U2_text = widgets.FloatText(layout=Layout(width="200px"),
                                         description = r"$U_{2}$",
                                         value = 1,
                                         disabled=True)
        self.U2_unit  = widgets.Label(value="V")
        self.U2_group = HBox([self.U2_text, self.U2_unit])

        self.Uint_slider = widgets.IntSlider(min = 1,
                                             max = 2,
                                             value = U,
                                             description = r"$U$")
        self.Uint_slider.observe(self.redraw, "value")
        
        self.d_slider = widgets.IntSlider(min = 10,
                                          max = 20,
                                          value = d,
                                          description = r"$d$")
        self.d_slider.observe(self.redraw, "value")
        self.d_unit  = widgets.Label(value="cm")
        self.d_group = HBox([self.d_slider, self.d_unit])
        
        self.A_slider = widgets.IntSlider(min = 100,
                                          max = 200,
                                          value = A,
                                          description = r"$A$")
        self.A_slider.observe(self.redraw, "value")
        self.A_unit  = widgets.Label(value="cm²")
        self.A_group = HBox([self.A_slider, self.A_unit])
        
        #self.x_slider = widgets.FloatSlider(min = 0,
        #                                    max = 1,
        #                                    value = 1,
        #                                    step = 0.01,
        #                                    description = r"$A_{\varepsilon} / A$")
        #self.x_slider.observe(self.redraw)
        
        self.sp_radio = widgets.RadioButtons(options=[["homogen", 1], ["geschichtet, seriell (in Reihe)", 2], ["geschichtet, parallel", 3]],
                                             description="Dielektrikum:",
                                             value=1)
        self.sp_radio.observe(self.redraw, "value")
        
        self.ED_radio = widgets.RadioButtons(options=[["E-Feld", 1], ["D-Feld", 2]],
                                             description="Feldlinien:",
                                             value=1)
        self.ED_radio.observe(self.redraw, "value")
        
        self.eps_slider = widgets.IntSlider(value = eps,
                                            min = 1,
                                            max = 4,
                                            description = r"$\varepsilon_{r}$")
        self.eps_slider.observe(self.redraw, "value")
        
        
        self.draw_background()
        self.redraw()
        
        if self.dtype == 1:   # Einfluss der Geometrie
            display(VBox([self.U_group,
                          self.d_group,
                          self.A_group]))
            
        elif self.dtype == 2: # Einfluss des Dielektrikums
            display(VBox([self.eps_slider,
                          self.sp_radio,
                          self.ED_radio]))
            
        elif self.dtype == 3: # Isolierte Anschlussklemmen
            display(VBox([self.U2_group,
                          self.d_group,
                          self.eps_slider,
                          self.ED_radio]))
    
        elif self.dtype == 0: # Testbetrieb / alle Regler zugänglich
            display(VBox([self.U_group,
                          self.d_group,
                          self.A_group,
                          self.sp_radio,
                          self.ED_radio,
                          self.eps_slider]))
            
            
    
    def draw_arrow(self, layer, xy, angle, length, color="#000000", line_width=2, tip_width=10, hold=True):
        
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)

        if length < 15:
            tip_length = 15 + (length - 15)
            width = tip_width / 2 * length / 15
        else:
            tip_length = 15
            width = tip_width / 2
        line_end = (xy[0] + (length - tip_length) * sin_a,
                    xy[1] + (length - tip_length) * cos_a)        
        
        with hold_canvas(layer):
            
            # Pfeilschaft
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180)
                          )
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180)
                          )
            layer.fill()
        
    
    
    def draw_background(self):
        
        bg1 = self.c["bg1"]
        bg1.line_width = 2
        bg1.stroke_style = colors["black"]
        bg1.fill_style = colors["white"]
        vsm = self.c["Vsource/meter"]
        
        with hold_canvas(bg1):
            bg1.begin_path()
            bg1.move_to( 25, 180)
            bg1.line_to( 25, 375)
            bg1.line_to(375, 375)
            bg1.line_to(375, 180)
            bg1.stroke()
        
        if self.dtype != 3:
            vsm.draw_image(img_vs, 175, 350)
        else:
            vsm.draw_image(img_vm, 175, 350)
            
        
        #bg.fill_arc(25, 180, 5, 0, 2 * pi)
        #bg.stroke_arc(25, 180, 5, 0, 2 * pi)
        #bg.fill_arc(375, 180, 5, 0, 2 * pi)
        #bg.stroke_arc(375, 180, 5, 0, 2 * pi)
        
    
    
    def draw_capacitor(self, *args, **kwarg):
        
        dist = self.d_slider.value * 3
        area = self.A_slider.value * 0.7
        
        cap = self.c["cap"]
        cap.line_width = 2
        cap.stroke_style = colors["black"]
        cap.fill_style = colors["lgrey"]
        
        with hold_canvas(cap):
            cap.begin_path()
            cap.move_to(180 - dist, 180 + area)
            cap.line_to(180 - dist, 180 - area)
            cap.line_to(200 - dist, 180 - area)
            cap.line_to(200 - dist, 180 + area)
            cap.close_path()
            cap.clear()
            cap.stroke()
            cap.fill()
            cap.stroke()
            cap.draw_image(cap, 2 * dist + 20)
        
        
        bg2 = self.c["bg2"]
        bg2.line_width = 2
        bg2.begin_path()
        
        with hold_canvas(bg2):
            bg2.move_to(25, 180)
            bg2.line_to(200 - dist, 180)
            bg2.clear()
            bg2.stroke()
            bg2.begin_path()
            bg2.move_to(375, 180)
            bg2.line_to(200 + dist, 180)
            bg2.stroke()
        
        
        
    def redraw(self, *args, **kwarg):
        
        self.draw_charges()
        self.draw_capacitor()
        self.draw_dielectricum()
        self.U2_text.value = 0.05 * self.d_slider.value / self.eps_slider.value
        
    
    
    def draw_hatched_box(self, canvas, x, y, lx, ly, color="#99ff99", gap=10, invert=False, line_width=2):
        
        x0 = x
        xi = x0
        y0 = y if not invert else y + ly
        yi = y0
        corr = -1 if invert else 1

        tmp = canvas
        tmp.stroke_style = color
        tmp.line_width = line_width
            
        if not invert:
            while xi < (x0 + lx):
                tmp.begin_path()
                tmp.move_to(xi, yi)
                if xi + ly < x0 + lx:
                    tmp.line_to(xi + ly, yi + ly * corr)
                else:
                    tmp.line_to(x0 + lx, (ly - (xi + ly - x0 - lx)) * corr + yi)
                tmp.stroke()

                xi += gap

            yi += gap * corr
            while yi < y0 + ly * corr:
                tmp.begin_path()
                tmp.move_to(x0, yi)
                if yi + lx < y0 + ly * corr:
                    tmp.line_to(x0 + lx, lx * corr + yi)
                else:
                    tmp.line_to(x0 + (lx - (yi + lx - y0 - ly)), y0 + ly)
                tmp.stroke()

                yi += gap * corr
        
        else:
            while xi < (x0 + lx):
                tmp.begin_path()
                tmp.move_to(xi, yi)
                if xi + ly < x0 + lx:
                    tmp.line_to(xi + ly, yi + ly * corr)
                else:
                    tmp.line_to(x0 + lx, (ly - (xi + ly - x0 - lx)) * corr + yi)
                tmp.stroke()

                xi += gap

            yi += gap * corr
            while yi > y0 + ly * corr:
                tmp.begin_path()
                tmp.move_to(x0, yi)
                if yi + ly * corr < y0 + ly * corr:
                    #print("1")
                    tmp.line_to(x0 + 0.72 * yi, y0 - ly)
                else:
                    print("2")
                    ##tmp.line_to(x0 + (lx - (yi + lx - y0 - ly)), y0 + ly)
                tmp.stroke()

                yi += gap * corr

        with hold_canvas(canvas):
            canvas.draw_image(tmp)
    
    
    
    def draw_dielectricum(self):
        
        A    = self.A_slider.value * 0.7
        d    = self.d_slider.value * 3
        ft   = self.sp_radio.value
        eps  = self.eps_slider.value
        diel = self.c["diel"]
        
        if ft == 1: # homogen
            diel.clear()
            if eps > 1:
                with hold_canvas(diel):
                    self.draw_hatched_box(diel, 200 - d, 180 - A, 2 * d, 2 * A, gap = 30 / (eps))
        
        elif ft == 2: # geschichtet, seriell
            diel.clear()
            if eps > 1:
                with hold_canvas(diel):
                    self.draw_hatched_box(diel, 200, 180 - A, d, 2 * A, gap = 30 / (eps))
        
        elif ft == 3: # geschichtet, parallel
            diel.clear()
            if eps > 1:
                with hold_canvas(diel):
                    self.draw_hatched_box(diel, 200 - d, 180, 2 * d, A, gap = 30 / (eps))
    
    
    
    def draw_charges(self):
        
        d   = float(self.d_slider.value) * 3
        A   = float(self.A_slider.value) * 0.7
        U   = float(self.U_slider.value) if self.dtype != 2 else self.Uint_slider.value
        E   = U / d * 2
        if self.dtype == 1: E *= 4
        if self.dtype == 2: E *= 2
        dt  = self.sp_radio.value
        ft  = self.ED_radio.value
        eps = self.eps_slider.value
        
        pos = self.c["pos"]
        neg = self.c["neg"]
        arr = self.c["field"]
        
        if E > 0:
            
            # Temporäre Canvas erstellen
            tmp_p = Canvas()
            tmp_n = Canvas()
            tmp_a = Canvas()
            
            # Maximale Höhe
            hmax = 2 * A - 20
            
            # Anzahl zu zeichnender Ladungsträger, nc = number of charges
            pf = 18 * (20 / 3 * E) * (A / 140) / 4 * eps if self.dtype != 3 else 8
            nc = int(pf)
            
            # Abstand zwischen Ladungsträgern, dy
            dy = round(hmax / pf) + 1
            
            # y-Offset für Symmetrie
            y0 = (hmax - (nc - 1) * dy) / 2
            
            self.c["field"].clear()
            for i in range(nc):
                
                # parallel geschichtet -> erster Ladungsträger mittig in oberer Hälfte
                if dt == 3 and i < 4:
                    if i == 0:
                        y = 74
                    if i == 1:
                        y = 140
                    if i == 2:
                        y = 206
                    if i == 3:
                        y = 272
                else:
                    y = 183 + y0 - A + i * dy
                    
                # bei parallel nur ersten in oberer Hälfte zeichnen
                if dt != 3 or \
                (dt == 3 and ((i >= nc / 2 or i < U) or \
                (eps == 1))):
                    
                    # Ladungsträger zeichnen
                    tmp_p.draw_image(img_pos, 183 - d, y)
                    tmp_n.draw_image(img_neg, 203 + d, y)
                    
                    # Pfeile zeichnen
                    
                    # homogen
                    # E: 1 pro eps
                    # D: 1 pro Ladungsträger
                    if dt == 1 and (i % eps == 0 or ft == 2):
                        self.draw_arrow(        tmp_a, (197 - d, y + 7), 0, 2 * (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)

                    # seriell geschichtet
                    if dt == 2:
                        if ft == 1 and eps > 1:
                            # E in Luft: 1 pro Ladungsträger (halbe Länge)
                            self.draw_arrow(    tmp_a, (197 - d, y + 7), 0,     (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)
                            # E in Dielektrikum: 1 pro eps (halbe Länge)
                            if i % eps == 0:
                                self.draw_arrow(tmp_a, (196    , y + 7), 0,     (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)
                        # D: pro Ladungsträger ein Pfeil (volle Länge)
                        else:
                            self.draw_arrow(    tmp_a, (197 - d, y + 7), 0, 2 * (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)

                
                # parallel geschichtet
                # E in Luft: 1 pro Ladungsträger
                # E in Dielektrikum: alle eps
                if dt == 3:
                    if ft == 2 and (i >= nc / 2 or i < 2):
                        self.draw_arrow(    tmp_a, (197 - d, y + 7), 0, 2 * (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)
                    if ft == 1 and i < 4:
                        self.draw_arrow(    tmp_a, (197 - d, y + 7), 0, 2 * (d - 1) + 8, color=colors["I"], line_width=2, tip_width=8)
                
            with hold_canvas(pos):
                pos.clear()
                pos.draw_image(tmp_p)

            with hold_canvas(neg):
                neg.clear()
                neg.draw_image(tmp_n)
            
            with hold_canvas(arr):
                arr.clear()
                arr.draw_image(tmp_a)
        
        else:
            pos.clear()
            neg.clear()
            arr.clear()



#K1 = Kondensator(U=1.5, d=15, A=150, eps=1, dtype=1)
#K2 = Kondensator(U=2, d=20, A=200, eps=2, dtype=2)
#K3 = Kondensator(U=2, d=20, A=200, eps=2, dtype=3)

## Konstante Spannung

$$
\begin{split}
E & = \frac{U}{d} \\ \\
D & = \varepsilon_{0} \cdot \varepsilon_{r} \cdot E \\ \\
Q & = D \cdot A
\end{split}
$$

### Einfluss der Geometrie

In [3]:
K1 = Kondensator(U=1.5, d=15, A=150, eps=1, dtype=1)

MultiCanvas(height=425, width=400)

### Einfluss des Dielektrikums

#### Reihenschaltung von Dielektrika

$$
\begin{split}
\frac{1}{\varepsilon_{ges}} & = \frac{1}{\varepsilon_{1}} + \frac{1}{\varepsilon_{2}} = \frac{\varepsilon_{1} \cdot \varepsilon_{2}}{\varepsilon_{1} + \varepsilon_{2}} \\ \\
\rightarrow \frac{1}{C_{ges}} & = \frac{1}{C_{1}} + \frac{1}{C_{2}} = \frac{C_{1} \cdot C_{2}}{C_{1} + C_{2}}
\end{split}
$$

#### Parallelschaltung von Dielektrika

$$
\begin{split}
\varepsilon_{ges} & = \varepsilon_{1} + \varepsilon_{2} \\ \\
\rightarrow C_{ges} & = C_{1} + C_{2}
\end{split}
$$

In [4]:
K2 = Kondensator(U=2, d=20, A=200, eps=2, dtype=2)

MultiCanvas(height=425, width=400)

## Konstante Ladung

Ausgangsbedingungen:

$$
\begin{split}
U_{1} & = 1\, \mathrm{V}\\
d_{1} & = 20\, \mathrm{mm}\\
\varepsilon_{r,1} & = 2 \\ \\
Q_{1} & = Q_{2} \\
\rightarrow U_{2} & = U_{1} \cdot \frac{\varepsilon_{r,1} \cdot d_{2}}{\varepsilon_{r,2} \cdot d_{1}}
\end{split}
$$

In [5]:
K3 = Kondensator(U=2, d=20, A=200, eps=1, dtype=3)
K3.d_slider.max = 40

MultiCanvas(height=425, width=400)